## 17. **Data augmentation**

- 데이터가 부족할 때, 새로운 데이터를 직접 수집하는 것은 **비용과 시간이 많이 소요**됨
    
    ⇒ 이를 보완하기 위해 **Data Augmentation(데이터 증강)** 기법을 사용함
    
- **하나의 이미지를 다양하게 변형(회전, 확대, 좌우반전 등)**시켜서, 마치 여러 장의 데이터를 가진 것처럼 학습시킴

In [1]:
import torch

# CPU/GPU 선택
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# train시에는 epoch마다 transform이 이미지를 무작위로 변형
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(30),
    transforms.RandomCrop(32, padding=4),
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# CIFAR-10은 10개의 클래스에 걸쳐 총 60,000개의 32x32 컬러 이미지로 구성된 데이터셋
train_dataset = datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=train_transform
)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=8,
)

test_dataset = datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=test_transform
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=False,
    num_workers=8,
)

print("Train dataset size:", len(train_dataset))
print("Validation dataset size:", len(test_dataset))


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Train dataset size: 50000
Validation dataset size: 10000


In [23]:
class ModernGAPCNN(nn.Module):
  def __init__(self, num_classes=10):
    super().__init__()

    self.feature_extractor = nn.Sequential(
        nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=1), # 16 * 30 * 30
        # channel을 기준으로 batch 정규화
        nn.BatchNorm2d(16),
        nn.ReLU(inplace=True),
        # stride를 2로 설정하여 feature map 크기를 줄임(pooling 대체)
        nn.Conv2d(16, 32, kernel_size=5, stride=2, padding=1), # 32 * 15 * 15
        nn.BatchNorm2d(32),
        nn.ReLU(inplace=True),
        nn.Conv2d(32, 64, kernel_size=5, stride=2, padding=1), # 64 * 7 * 17
        nn.BatchNorm2d(64),
        nn.ReLU(inplace=True),
    )

    # Global Average Pooling을 이용해 Channel별로 전체 값을 평균내어 1개의 값으로 축소
    self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1)) # 64 × 1 × 1
    self.classifier = nn.Linear(64, num_classes)

  def forward(self, x):
      x = self.feature_extractor(x)
      x = self.global_avg_pool(x)
      x = torch.flatten(x, 1)
      x = self.classifier(x)
      return x


In [24]:
import torch.optim as optim

net = ModernGAPCNN(10)
net.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

for epoch in range(10):
  net.train()
  for batch_idx, (data, label) in enumerate(train_loader):
    data, label = data.to(device), label.to(device)
    optimizer.zero_grad()
    output = net(data)
    train_loss = loss_fn(output, label)
    train_loss.backward()
    optimizer.step()

  net.eval()
  val_loss = 0
  correct = 0
  with torch.no_grad():
    for data, label in test_loader:
      data, label = data.to(device), label.to(device)
      output = net(data)
      val_loss += loss_fn(output, label).item()
      pred = output.argmax(dim=1, keepdim=True)
      correct += pred.eq(label.view_as(pred)).sum().item()

  val_loss /= len(test_loader.dataset)
  accuracy = 100. * correct / len(test_loader.dataset)

  print(f'Epoch {epoch+1}, Loss: {train_loss.item():.4f}, Val Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%')

Epoch 1, Loss: 1.6211, Val Loss: 0.2219, Accuracy: 34.55%
Epoch 2, Loss: 1.7361, Val Loss: 0.2026, Accuracy: 41.10%
Epoch 3, Loss: 1.9358, Val Loss: 0.2028, Accuracy: 41.53%
Epoch 4, Loss: 2.1931, Val Loss: 0.1921, Accuracy: 45.04%
Epoch 5, Loss: 1.5344, Val Loss: 0.1955, Accuracy: 43.35%
Epoch 6, Loss: 1.2317, Val Loss: 0.1871, Accuracy: 47.34%
Epoch 7, Loss: 0.9234, Val Loss: 0.1770, Accuracy: 49.36%
Epoch 8, Loss: 1.7471, Val Loss: 0.1737, Accuracy: 50.31%
Epoch 9, Loss: 1.5558, Val Loss: 0.1769, Accuracy: 50.46%
Epoch 10, Loss: 1.5857, Val Loss: 0.1733, Accuracy: 50.84%
